In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [0]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [0]:
#lookup tables


import numpy as np
import problem_unittests as tests
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab=set(text)

    #words=text.split()
    #counts=Counter(words)
    #vocab=sorted(counts,key=counts.get,reverse=True)
    vocab_to_int={word:ii for ii ,word in enumerate(vocab)}
    int_to_vocab={ii:word for ii ,word in enumerate(vocab)}
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)


Tests Passed


In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    toke={'.':'||Period||',
            ',':'||Comma||',
            '"':'||Quotation_Mark||',
            ';':'||Semicolon||', 
            '!':'||Exclamation_Mark||',
            '?':'||Question_Mark||', 
            '(':'||Left_Parentheses||', 
            ')':'||Right_Parentheses||',
            '--':'||Dash||',
            '\n':'||Return||'}
    return toke

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)


Tests Passed


In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data

helper.preprocess_and_save_data(data_dir,token_lookup,create_lookup_tables)

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.13.1
Default GPU Device: /device:GPU:0


In [0]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_text=tf.placeholder(tf.int32,shape=[None,None],name='input')
    output_target=tf.placeholder(tf.int32,shape=[None,None],name='targets')
    learning_rate=tf.placeholder(tf.float32,name='learning_rate')
    
    return (input_text, output_target, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)


Tests Passed


In [0]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    def bulid_cell(rnn_size):
      lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
      
    # Add dropout to the cell outputs
      #drop = tf.contrib.rnn.DropoutWrapper(lstm,keep_prob)
      return lstm
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([bulid_cell(rnn_size) for _ in range(3)])
    initial_state = cell.zero_state(batch_size,tf.float32)
    
    initial_state=tf.identity(initial_state,name='initial_state')
    return (cell , initial_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


In [0]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding=tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    embed=tf.nn.embedding_lookup(embedding,input_data)
    return embed
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)


Tests Passed


In [0]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    Outputs,final_state=tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    final_state=tf.identity(final_state,name='final_state')
    return( Outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


In [0]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embedding_layer=get_embed(input_data,vocab_size,embed_dim)
    outputs, final_state=build_rnn(cell,embedding_layer)
    Logits=tf.layers.dense(outputs,vocab_size,activation=None,use_bias=True)
    return (Logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


In [0]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # Calculate the number of batches
    num_batches = len(int_text) // (batch_size  * seq_length)
    # Drop long batches. Transform into a numpy array and reshape it for our purposes
    np_text = np.array(int_text[:num_batches * (batch_size  * seq_length)])
    # Reshape the data to give us the inputs sequence.
    in_text = np_text.reshape(-1, seq_length)
    # Roll (shift) and reshape to get target sequences (maybe not optimal)
    tar_text = np.roll(np_text, -1).reshape(-1, seq_length)
    output = np.zeros(shape=(num_batches, 2, batch_size, seq_length), dtype=np.int)
    # Prepare the output
    for idx in range(0, in_text.shape[0]):
        jj = idx % num_batches
        ii = idx // num_batches
        output[jj,0,ii,:] = in_text[idx,:]
        output[jj,1,ii,:] = tar_text[idx,:]
    return output
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)


Tests Passed


In [0]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 16
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 100
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save1'

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/33   train_loss = 8.821
Epoch   3 Batch    1/33   train_loss = 5.920
Epoch   6 Batch    2/33   train_loss = 5.891
Epoch   9 Batch    3/33   train_loss = 5.943
Epoch  12 Batch    4/33   train_loss = 5.926
Epoch  15 Batch    5/33   train_loss = 6.064
Epoch  18 Batch    6/33   train_loss = 5.950
Epoch  21 Batch    7/33   train_loss = 5.977
Epoch  24 Batch    8/33   train_loss = 5.802
Epoch  27 Batch    9/33   train_loss = 5.420
Epoch  30 Batch   10/33   train_loss = 5.155
Epoch  33 Batch   11/33   train_loss = 4.677
Epoch  36 Batch   12/33   train_loss = 4.661
Epoch  39 Batch   13/33   train_loss = 4.491
Epoch  42 Batch   14/33   train_loss = 4.367
Epoch  45 Batch   15/33   train_loss = 4.161
Epoch  48 Batch   16/33   train_loss = 4.075
Epoch  51 Batch   17/33   train_loss = 3.990
Epoch  54 Batch   18/33   train_loss = 3.885
Epoch  57 Batch   19/33   train_loss = 3.667
Epoch  60 Batch   20/33   train_loss = 3.642
Epoch  63 Batch   21/33   train_loss = 3.596
Epoch  66 

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [0]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state_tensor = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state_tensor = loaded_graph.get_tensor_by_name('final_state:0')
    probs_tensor = loaded_graph.get_tensor_by_name('probs:0')
    return (input_tensor, initial_state_tensor, final_state_tensor, probs_tensor)




"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)


Tests Passed


In [0]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    argmax_pr = np.argmax(probabilities)
    next_word = int_to_vocab[argmax_pr]
    return next_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)


Tests Passed


In [98]:
gen_length = 500
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save1
homer_simpson:(loud) did you ever see that" blue man group?"
duffman:(to moe) huh. i'm sweeter weird comin' and please.
moe_szyslak: yeah, i didn't say that.
moe_szyslak: oh what, that old beautiful, homer. i don't don't know what how it. i came to the springfield thing we got from the beach from the way!
moe_szyslak:(waking-up voice.(then) oh my god, you're dead.
moe_szyslak: what are you really are a little more of this best lady. i'm not just do the same, and we feel too to the drug customers.
barney_gumble:(sighs) i can't drink i haven't seen that.
moe_szyslak:(sincere) what's the genius, homer!
homer_simpson:(toasting) you're almost out," dear midge:.", i'll got away three says" and then he been things as by her!
lisa_simpson:(increasingly arm noises) we callin' a problem in our japanese stick too filthy has an ice kick this back.
homer_simpson: well, i can't tell my english fat, i can spend the car again?
lenny_leonard: and you kn